In [424]:
import pandas as pd
import json
import requests
from PIL import Image
import os
import time
from io import BytesIO
from tqdm import tqdm

1. Read the csv of key artworks
2. Keep only data fields of interest
3. Save as a Python dataframe

In [425]:
df = pd.read_csv('raw_data/artic-api-data/getting-started/someArtworks.csv')

In [426]:
df

,id,title,main_reference_number,department_title,artist_title
0,869,The Watermill with the Great Red Roof,1894.1031,Painting and Sculpture of Europe,Meindert Hobbema
1,2189,Ready-to-Wear,1956.137,Arts of the Americas,Stuart Davis
2,2816,Interior at Nice,1956.339,Modern Art,Henri Matisse
3,4081,Gian Lodovico Madruzzo,1929.912,Painting and Sculpture of Europe,Giovanni Battista Moroni
4,4428,Mère Grégoire,1930.78,Painting and Sculpture of Europe,Gustave Courbet
...,...,...,...,...,...
418,234004,Untitled 72–12–A,2016.62,Contemporary Art,"Chung, Sang-Hwa"
419,234433,Involvement Series,2016.66,Contemporary Art,Wanda Pimentel
420,234781,Christ Carrying the Cross,2016.15,Painting and Sculpture of Europe,Sebastiano del Piombo
421,234972,Small Town by Day (Badische Kleinstadt bei Tage),2016.46,Modern Art,Georg Scholz


In [427]:
df.iloc[0]['id']

869

In [428]:
with open("raw_data/artic-api-data/json/artworks/869.json",'r') as f:
    data = json.loads(f.read())
data

{'id': 869,
 'api_model': 'artworks',
 'api_link': 'https://api.artic.edu/api/v1/artworks/869',
 'is_boosted': True,
 'title': 'The Watermill with the Great Red Roof',
 'alt_titles': None,
 'thumbnail': {'lqip': '',
  'width': 9443,
  'height': 6967,
  'alt_text': 'Landscape painting featuring a wavy tree tree in the foreground and a watermill with a red roof adjacent to a river at right. Colors are subdued earth tones, the foliage a dark green and the river a muddy brown. Water flows through the mill wheel under a cloudy sky of gray and blue. Three individuals can be seen in the distance at left.'},
 'main_reference_number': '1894.1031',
 'has_not_been_viewed_much': False,
 'boost_rank': None,
 'date_start': 1665,
 'date_end': 1675,
 'date_display': '1662/65',
 'date_qualifier_title': '',
 'date_qualifier_id': None,
 'artist_display': 'Meindert Hobbema\nDutch, 1638–1709',
 'place_of_origin': 'Holland',
 'dimensions': '32 × 43 1/4 in. (81.3 × 110 cm)',
 'medium_display': 'Oil on canvas

In [429]:
keys = ['id', 'title', 'thumbnail', 'date_start', 'date_end', 'date_display',
       'artist_display', 'place_of_origin', 'medium_display', 'is_public_domain',
        'copyright_notice', 'artist_id', 'artist_title', 'term_titles',
        'image_id', 'document_ids', 'sound_ids', 'video_ids','text_ids']

In [430]:
artworks_df = pd.DataFrame()
for index, value in df.iterrows():
    id = df.iloc[index]['id']
    with open(f'raw_data/artic-api-data/json/artworks/{id}.json','r') as f:
        data = json.loads(f.read())
        artwork_dict = {key: data[key] for key in keys}
        artwork_df = pd.DataFrame.from_dict(artwork_dict, orient='index')
        artworks_df = pd.concat([artworks_df, artwork_df.T])

In [431]:
artworks_df = artworks_df.reset_index()
artworks_df = artworks_df.drop(columns='index')
artworks_df

,id,title,thumbnail,date_start,date_end,date_display,artist_display,place_of_origin,medium_display,is_public_domain,copyright_notice,artist_id,artist_title,term_titles,image_id,document_ids,sound_ids,video_ids,text_ids
0,869,The Watermill with the Great Red Roof,"{'lqip': 'data:image/gif;base64,R0lGODlhBwAFAP...",1665,1675,1662/65,"Meindert Hobbema\nDutch, 1638–1709",Holland,Oil on canvas,True,None,34954,Meindert Hobbema,"[oil on canvas, Century of Progress, Realism, ...",237c25a2-6051-a8e7-1610-a01938d4deab,[fd2bd2c7-fb34-6e84-034a-49cb05ec4d6d],[fd2bd2c7-fb34-6e84-034a-49cb05ec4d6d],[],[]
1,2189,Ready-to-Wear,"{'lqip': 'data:image/gif;base64,R0lGODlhBAAFAP...",1955,1955,1955,"Stuart Davis\nAmerican, 1892–1964",United States,Oil on canvas,False,None,34155,Stuart Davis,"[painting, oil paint (paint), Modernism, oil p...",8a4e78cf-d284-f262-6c9d-0ad15bc2d43b,[],[],[],[]
2,2816,Interior at Nice,"{'lqip': 'data:image/gif;base64,R0lGODlhAwAFAP...",1919,1920,1919 or 1920,"Henri Matisse\nFrench, 1869–1954",France,Oil on canvas,False,© 2018 Succession H. Matisse / Artists Rights ...,35670,Henri Matisse,"[painting, oil painting, oil paint (paint), in...",2193cdda-2691-2802-0776-145dee77f7ea,[],[],[],[]
3,4081,Gian Lodovico Madruzzo,"{'lqip': 'data:image/gif;base64,R0lGODlhAwAFAP...",1551,1552,1551/52,"Giovanni Battista Moroni\nItalian, 1520/24-1578",Italy,Oil on canvas,True,None,35838,Giovanni Battista Moroni,"[oil on canvas, Century of Progress, animals, ...",bc24fe7f-4171-c785-bff2-b2ed3777a47d,[1faeb566-bc3a-9792-97f4-4ef156bd924d],[1faeb566-bc3a-9792-97f4-4ef156bd924d],[],[]
4,4428,Mère Grégoire,"{'lqip': 'data:image/gif;base64,R0lGODlhBAAFAP...",1855,1859,1855 and 1857/59,"Gustave Courbet\nFrench, 1819-1877",France,Oil on canvas,True,None,34049,Gustave Courbet,"[painting, oil painting, oil paint (paint), Ce...",2e6d08c8-dac0-4f1c-3e56-de3539ceb905,[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,234004,Untitled 72–12–A,"{'lqip': 'data:image/gif;base64,R0lGODlhBAAFAP...",1972,1972,1972,"Chung Sang-Hwa\nKorean, born 1932",None,Acrylic on canvas,False,None,114644,"Chung, Sang-Hwa","[painting (image making), acrylic paint, patte...",86f8596f-6fc2-5f28-dc26-8526b5da2290,[],[],[],[]
419,234433,Involvement Series,"{'lqip': 'data:image/gif;base64,R0lGODlhBAAFAP...",1968,1968,1968,"Wanda Pimentel\nBrazilian, 1943-2019",None,Vinyl on canvas,False,None,114672,Wanda Pimentel,"[synthetic resin paint, clothing, painting, co...",0f6ffedf-d080-9528-9543-31087899ce57,[],[],[],[]
420,234781,Christ Carrying the Cross,"{'lqip': 'data:image/gif;base64,R0lGODlhBAAFAP...",1515,1517,about 1515/1517,"Sebastiano del Piombo \nItalian, about 1485–1547",None,Oil on panel,True,None,41499,Sebastiano del Piombo,"[painting, oil painting, oil paint (paint), Je...",13a251ba-2c7a-2d0c-3f65-820c0e1f1a39,[],[],[],[]
421,234972,Small Town by Day (Badische Kleinstadt bei Tage),"{'lqip': 'data:image/gif;base64,R0lGODlhBAAFAP...",1922,1923,1922–23,"Georg Scholz\nGerman, 1890–1945",None,Oil on board,False,None,57509,Georg Scholz,"[oil painting, oil paint (paint), towns, paint...",cd773e50-2f0f-9d5d-e112-a5271164abf2,[],[],[],[]


* See how many works are in the public domain
* See how many works have sound files

In [432]:
artworks_df.loc[0]['thumbnail']['alt_text']

'Landscape painting featuring a wavy tree tree in the foreground and a watermill with a red roof adjacent to a river at right. Colors are subdued earth tones, the foliage a dark green and the river a muddy brown. Water flows through the mill wheel under a cloudy sky of gray and blue. Three individuals can be seen in the distance at left.'

In [433]:
artworks_df['is_public_domain'].value_counts(normalize=True)

True     0.531915
False    0.468085
Name: is_public_domain, dtype: float64

In [434]:
artworks_df[artworks_df['image_id'].isnull()]

,id,title,thumbnail,date_start,date_end,date_display,artist_display,place_of_origin,medium_display,is_public_domain,copyright_notice,artist_id,artist_title,term_titles,image_id,document_ids,sound_ids,video_ids,text_ids
371,186392,Wall Drawing #821: A black square divided hori...,None,1997,1997,1997,"Sol LeWitt\nAmerican, 1928-2007",United States,Acrylic paint,False,© 2008 The Estate of Sol LeWitt,35429,Sol LeWitt,"[20th Century, painting (image making), acryli...",None,[],[],[],[]


In [435]:
artworks_df['sound_ids'].value_counts(normalize=True)

[]                                        0.671395
[27bdd742-63a9-0a04-1514-efe1b99e70e7]    0.002364
[50468293-95f8-b028-99a2-5de3131b323e]    0.002364
[37f1444f-6c1f-dde3-98fe-dece769ff35d]    0.002364
[7fcd707c-97e9-67b2-07f6-9f8d7a96d9cd]    0.002364
                                            ...   
[679e5661-8cbe-e465-d228-59203b8f0425]    0.002364
[6065bb79-2319-0414-fa02-304ec5f72771]    0.002364
[02e976d2-8c4a-3cc2-9d2b-3cee40fe8291]    0.002364
[ad4b851f-f04b-694b-e3c1-73166263775b]    0.002364
[dd9fbf6c-126a-201d-9462-245fbe34e117]    0.002364
Name: sound_ids, Length: 140, dtype: float64

* Extract alt_text only

In [436]:
artworks_df.loc[371]

id                                                             186392
title               Wall Drawing #821: A black square divided hori...
thumbnail                                                        None
date_start                                                       1997
date_end                                                         1997
date_display                                                     1997
artist_display                        Sol LeWitt\nAmerican, 1928-2007
place_of_origin                                         United States
medium_display                                          Acrylic paint
is_public_domain                                                False
copyright_notice                      © 2008 The Estate of Sol LeWitt
artist_id                                                       35429
artist_title                                               Sol LeWitt
term_titles         [20th Century, painting (image making), acryli...
image_id            

In [437]:
artworks_df.loc[371]['thumbnail']['alt_text']

TypeError: 'NoneType' object is not subscriptable

In [438]:
for index, value in artworks_df.iterrows():
    if index != 371:
        artworks_df.loc[index, ['alt_text']] = artworks_df.loc[index]['thumbnail']['alt_text']

In [439]:
artworks_df

,id,title,thumbnail,date_start,date_end,date_display,artist_display,place_of_origin,medium_display,is_public_domain,copyright_notice,artist_id,artist_title,term_titles,image_id,document_ids,sound_ids,video_ids,text_ids,alt_text
0,869,The Watermill with the Great Red Roof,"{'lqip': 'data:image/gif;base64,R0lGODlhBwAFAP...",1665,1675,1662/65,"Meindert Hobbema\nDutch, 1638–1709",Holland,Oil on canvas,True,None,34954,Meindert Hobbema,"[oil on canvas, Century of Progress, Realism, ...",237c25a2-6051-a8e7-1610-a01938d4deab,[fd2bd2c7-fb34-6e84-034a-49cb05ec4d6d],[fd2bd2c7-fb34-6e84-034a-49cb05ec4d6d],[],[],Landscape painting featuring a wavy tree tree ...
1,2189,Ready-to-Wear,"{'lqip': 'data:image/gif;base64,R0lGODlhBAAFAP...",1955,1955,1955,"Stuart Davis\nAmerican, 1892–1964",United States,Oil on canvas,False,None,34155,Stuart Davis,"[painting, oil paint (paint), Modernism, oil p...",8a4e78cf-d284-f262-6c9d-0ad15bc2d43b,[],[],[],[],"Large abstract painting with red, blue, white,..."
2,2816,Interior at Nice,"{'lqip': 'data:image/gif;base64,R0lGODlhAwAFAP...",1919,1920,1919 or 1920,"Henri Matisse\nFrench, 1869–1954",France,Oil on canvas,False,© 2018 Succession H. Matisse / Artists Rights ...,35670,Henri Matisse,"[painting, oil painting, oil paint (paint), in...",2193cdda-2691-2802-0776-145dee77f7ea,[],[],[],[],"Interior of warm colored room, woman sits on b..."
3,4081,Gian Lodovico Madruzzo,"{'lqip': 'data:image/gif;base64,R0lGODlhAwAFAP...",1551,1552,1551/52,"Giovanni Battista Moroni\nItalian, 1520/24-1578",Italy,Oil on canvas,True,None,35838,Giovanni Battista Moroni,"[oil on canvas, Century of Progress, animals, ...",bc24fe7f-4171-c785-bff2-b2ed3777a47d,[1faeb566-bc3a-9792-97f4-4ef156bd924d],[1faeb566-bc3a-9792-97f4-4ef156bd924d],[],[],A man with sparse brown facial hair stands in ...
4,4428,Mère Grégoire,"{'lqip': 'data:image/gif;base64,R0lGODlhBAAFAP...",1855,1859,1855 and 1857/59,"Gustave Courbet\nFrench, 1819-1877",France,Oil on canvas,True,None,34049,Gustave Courbet,"[painting, oil painting, oil paint (paint), Ce...",2e6d08c8-dac0-4f1c-3e56-de3539ceb905,[],[],[],[],Dark portrait of older woman in black dress wi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,234004,Untitled 72–12–A,"{'lqip': 'data:image/gif;base64,R0lGODlhBAAFAP...",1972,1972,1972,"Chung Sang-Hwa\nKorean, born 1932",None,Acrylic on canvas,False,None,114644,"Chung, Sang-Hwa","[painting (image making), acrylic paint, patte...",86f8596f-6fc2-5f28-dc26-8526b5da2290,[],[],[],[],"Abstracted colors of tan and brown, painted di..."
419,234433,Involvement Series,"{'lqip': 'data:image/gif;base64,R0lGODlhBAAFAP...",1968,1968,1968,"Wanda Pimentel\nBrazilian, 1943-2019",None,Vinyl on canvas,False,None,114672,Wanda Pimentel,"[synthetic resin paint, clothing, painting, co...",0f6ffedf-d080-9528-9543-31087899ce57,[],[],[],[],Dark blue shades create the background with a ...
420,234781,Christ Carrying the Cross,"{'lqip': 'data:image/gif;base64,R0lGODlhBAAFAP...",1515,1517,about 1515/1517,"Sebastiano del Piombo \nItalian, about 1485–1547",None,Oil on panel,True,None,41499,Sebastiano del Piombo,"[painting, oil painting, oil paint (paint), Je...",13a251ba-2c7a-2d0c-3f65-820c0e1f1a39,[],[],[],[],A work made of oil on panel.
421,234972,Small Town by Day (Badische Kleinstadt bei Tage),"{'lqip': 'data:image/gif;base64,R0lGODlhBAAFAP...",1922,1923,1922–23,"Georg Scholz\nGerman, 1890–1945",None,Oil on board,False,None,57509,Georg Scholz,"[oil painting, oil paint (paint), towns, paint...",cd773e50-2f0f-9d5d-e112-a5271164abf2,[],[],[],[],A work made of oil on board.


In [443]:
artworks_df.to_csv('raw_data/csv/artic_topartworks.csv', encoding='utf-8-sig')

* Create a subset of works in the public domain

In [444]:
public_df = artworks_df[artworks_df['is_public_domain'] == True]

In [445]:
public_df

,id,title,thumbnail,date_start,date_end,date_display,artist_display,place_of_origin,medium_display,is_public_domain,copyright_notice,artist_id,artist_title,term_titles,image_id,document_ids,sound_ids,video_ids,text_ids,alt_text
0,869,The Watermill with the Great Red Roof,"{'lqip': 'data:image/gif;base64,R0lGODlhBwAFAP...",1665,1675,1662/65,"Meindert Hobbema\nDutch, 1638–1709",Holland,Oil on canvas,True,None,34954,Meindert Hobbema,"[oil on canvas, Century of Progress, Realism, ...",237c25a2-6051-a8e7-1610-a01938d4deab,[fd2bd2c7-fb34-6e84-034a-49cb05ec4d6d],[fd2bd2c7-fb34-6e84-034a-49cb05ec4d6d],[],[],Landscape painting featuring a wavy tree tree ...
3,4081,Gian Lodovico Madruzzo,"{'lqip': 'data:image/gif;base64,R0lGODlhAwAFAP...",1551,1552,1551/52,"Giovanni Battista Moroni\nItalian, 1520/24-1578",Italy,Oil on canvas,True,None,35838,Giovanni Battista Moroni,"[oil on canvas, Century of Progress, animals, ...",bc24fe7f-4171-c785-bff2-b2ed3777a47d,[1faeb566-bc3a-9792-97f4-4ef156bd924d],[1faeb566-bc3a-9792-97f4-4ef156bd924d],[],[],A man with sparse brown facial hair stands in ...
4,4428,Mère Grégoire,"{'lqip': 'data:image/gif;base64,R0lGODlhBAAFAP...",1855,1859,1855 and 1857/59,"Gustave Courbet\nFrench, 1819-1877",France,Oil on canvas,True,None,34049,Gustave Courbet,"[painting, oil painting, oil paint (paint), Ce...",2e6d08c8-dac0-4f1c-3e56-de3539ceb905,[],[],[],[],Dark portrait of older woman in black dress wi...
5,4575,Judith,"{'lqip': 'data:image/gif;base64,R0lGODlhBAAFAP...",1495,1580,about 1540,"Jan Sanders van Hemessen\nNetherlandish, activ...",Netherlands,Oil on panel,True,None,11328,Jan Sanders van Hemessen,"[oil on panel, women, oil painting, oil paint ...",5b9758c9-24ef-8661-fac0-c9a4229eefd4,"[cb4b2557-071d-cb64-7195-2adc92592584, 28d1af9...",[28d1af93-d306-1886-94a0-6e4c8420fb8a],[],[cb4b2557-071d-cb64-7195-2adc92592584],"Painting of Judith, naked, holding sword, with..."
6,4758,Stoke-by-Nayland,"{'lqip': 'data:image/gif;base64,R0lGODlhBwAFAP...",1836,1836,1836,"John Constable\nEnglish, 1776-1837",England,Oil on canvas,True,None,34023,John Constable,"[oil on canvas, 19th century, oil painting, oi...",400ce9e8-2f67-44e2-dd68-e6c98880d27f,[86aaf438-63d2-52ea-f883-e81dac4ee695],[86aaf438-63d2-52ea-f883-e81dac4ee695],[],[],"Large English country landscape painting, farm..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,193664,The Captive Slave (Ira Aldridge),"{'lqip': 'data:image/gif;base64,R0lGODlhBAAFAP...",1827,1827,1827,"John Philip Simpson\nEnglish, 1782–1847",None,Oil on canvas,True,None,107195,John Philip Simpson,"[men, oil painting, oil paint (paint), paintin...",5c3cbb0f-03e6-7d46-60b6-f612ed71b5d3,[e694a130-f9f9-d8e0-d06b-0c922636f6df],[e694a130-f9f9-d8e0-d06b-0c922636f6df],[],[],Painting of a dark-skinned man seated and wear...
399,199854,Shaffron,"{'lqip': 'data:image/gif;base64,R0lGODlhBAAFAP...",1550,1600,c. 1575,"Italian, Milan",Italy,Steel with gilding,True,None,None,None,"[horses, gilding, steel, horse equipment, plan...",cac09e60-7c12-5088-3efa-e145b3b47c5b,[],[],[],[],"Steel horse helmet, brass trim, protruding spike."
401,210482,I'timad-ud-Daula's Tomb at Agra,"{'lqip': 'data:image/gif;base64,R0lGODlhBwAFAP...",1815,1825,c. 1820,India\nAgra,India,Watercolor on paper watermarked J. Whatman wit...,True,None,None,None,"[paper (fiber product), painting (image making...",36e70de8-0ba3-2fe5-dd2e-53612b48a395,[],[],[],[],A work made of watercolor on paper watermarked...
402,210511,A Monumental Portrait of a Monkey,"{'lqip': 'data:image/gif;base64,R0lGODlhBgAFAP...",1700,1715,c. 1705–1710,"India\nRajasthan, Mewar, Udaipur, attributed t...",India,Opaque watercolor and gold on paper,True,None,None,None,"[paper (fiber product), painting (image making...",67c7274f-612f-3180-0a12-e2c7ed5a7dd4,[],[],[],[],A work made of opaque watercolor and gold on p...


In [446]:
public_df.to_csv('raw_data/csv/artic_topartworks_public.csv', encoding='utf-8-sig')

* Save images to disk at a rate limit of 1 per second

In [371]:
output_folder = 'raw_data/img/artic_topartworks_public'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [372]:
for index, value in tqdm(public_df.iterrows()):
    id = public_df.loc[index]['id']
    identifier = public_df.loc[index]['image_id']
    url = f'https://www.artic.edu/iiif/2/{identifier}/full/843,/0/default.jpg'
    print(f'download {url}')
    response = requests.get(url)
    if response.status_code == 404:
        print(f'Skipping {url}: Image not found')
        continue
        
    # Convert the content to an image
    image = Image.open(BytesIO(response.content))

    # Save the image to the output folder with its id as unique filename
    filename = f'{id}.jpg'
    output_path = os.path.join(output_folder, filename)
    image.save(output_path)

    # Wait for 1 second before downloading the next image
    time.sleep(1)

0it [00:00, ?it/s]

download https://www.artic.edu/iiif/2/237c25a2-6051-a8e7-1610-a01938d4deab/full/843,/0/default.jpg


1it [00:01,  1.05s/it]

download https://www.artic.edu/iiif/2/bc24fe7f-4171-c785-bff2-b2ed3777a47d/full/843,/0/default.jpg


2it [00:02,  1.06s/it]

...

223it [03:59,  1.06s/it]

download https://www.artic.edu/iiif/2/67c7274f-612f-3180-0a12-e2c7ed5a7dd4/full/843,/0/default.jpg


224it [04:01,  1.06s/it]

download https://www.artic.edu/iiif/2/13a251ba-2c7a-2d0c-3f65-820c0e1f1a39/full/843,/0/default.jpg


225it [04:02,  1.08s/it]


* Download sound files

In [373]:
public_df['sound_ids'].value_counts(normalize=True)

[]                                                                              0.568889
[fd2bd2c7-fb34-6e84-034a-49cb05ec4d6d]                                          0.004444
[031e5d5f-5a02-7dc4-faba-5f46a7914039, 367d7d17-cf67-faca-eb21-8d898265336f]    0.004444
[34302929-1430-aa1c-8cca-23d94444ba2d]                                          0.004444
[bebc4218-ec27-d5e5-3755-3ab79420a5d1]                                          0.004444
                                                                                  ...   
[a0d232ce-be78-8c35-3ff4-8fc21665b010]                                          0.004444
[3861dbe0-e54b-7fdc-225c-3f192a7b3947, 07a1f837-9df3-6675-9992-248fbbb6a485]    0.004444
[4bbf187a-cef1-bec1-9f4d-9946d7bfb8b6, 5968010e-07a0-1055-072c-8fae9acfa8b8]    0.004444
[0f5293cb-626b-ee91-81f1-ed14623ce3fd, a8dabe2b-9c27-b59d-2964-f9f532fb785d]    0.004444
[e694a130-f9f9-d8e0-d06b-0c922636f6df]                                          0.004444
Name: sound_ids, Leng

In [405]:
public_df.iloc[2]

id                                                               4428
title                                                   Mère Grégoire
thumbnail           {'lqip': 'data:image/gif;base64,R0lGODlhBAAFAP...
date_start                                                       1855
date_end                                                         1859
date_display                                         1855 and 1857/59
artist_display                     Gustave Courbet\nFrench, 1819-1877
place_of_origin                                                France
medium_display                                          Oil on canvas
is_public_domain                                                 True
copyright_notice                                                 None
artist_id                                                       34049
artist_title                                          Gustave Courbet
term_titles         [painting, oil painting, oil paint (paint), Ce...
image_id            

In [408]:
folder_path = os.path.join(os.getcwd(), 'raw_data/sounds/artic_topartworks_public')
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

In [411]:
for index, value in public_df.iterrows():
    id = public_df.loc[index]['id']
    try:
        identifier = public_df.loc[index]['sound_ids'][0]
        url = f'https://api.artic.edu/api/v1/sounds/{identifier}'
        response = requests.get(url)

        if response.ok:
            json_data = response.json()
            content_uri = json_data['data']['content']
            print(content_uri)
            filename = f'{id}.mp3'
            full_path = os.path.join(folder_path, filename)
            with open(full_path, 'wb') as f:
                f.write(requests.get(content_uri).content)

            print(f'Sound saved to {filename}')
        else:
            print(f'Request failed with status code {response.status_code}')

    except:
        print('error encountered, skipping...')
        continue
    
    time.sleep(1)

https://www.artic.edu/assets/fd2bd2c7-fb34-6e84-034a-49cb05ec4d6d
Sound saved to 869.mp3
https://www.artic.edu/assets/1faeb566-bc3a-9792-97f4-4ef156bd924d
Sound saved to 4081.mp3
error encountered, skipping...
https://www.artic.edu/assets/28d1af93-d306-1886-94a0-6e4c8420fb8a
Sound saved to 4575.mp3
https://www.artic.edu/assets/86aaf438-63d2-52ea-f883-e81dac4ee695
Sound saved to 4758.mp3
error encountered, skipping...
https://www.artic.edu/assets/882a8a15-0b99-de46-a868-1e8ca0596a9d
Sound saved to 4788.mp3
error encountered, skipping...
https://www.artic.edu/assets/2364b23c-f4e0-ce4e-9d65-d31a6b0f5dff
Sound saved to 5848.mp3
error encountered, skipping...
https://www.artic.edu/assets/ee168f31-6695-e48c-e430-a57a4ba7fdce
Sound saved to 8958.mp3
https://www.artic.edu/assets/69b174c7-8039-b5c7-7474-957a2d0fee0e
Sound saved to 8991.mp3
error encountered, skipping...
error encountered, skipping...
error encountered, skipping...
https://www.artic.edu/assets/390c54e3-82bc-c57f-077a-6b5a9a1d939

https://www.artic.edu/assets/c278a88c-7e83-7a8e-7b35-6739f1014b46
Sound saved to 93900.mp3
https://www.artic.edu/assets/cbcafdef-b2aa-1efa-d634-e23d3febfec9
Sound saved to 95998.mp3
https://www.artic.edu/assets/3f30d0f8-f77c-06af-4adc-14078b397e1f
Sound saved to 97910.mp3
https://www.artic.edu/assets/04d3cbdd-415b-76ea-18b9-178cc683d620
Sound saved to 97916.mp3
error encountered, skipping...
error encountered, skipping...
error encountered, skipping...
error encountered, skipping...
https://www.artic.edu/assets/37f1444f-6c1f-dde3-98fe-dece769ff35d
Sound saved to 102591.mp3
https://www.artic.edu/assets/7fcd707c-97e9-67b2-07f6-9f8d7a96d9cd
Sound saved to 102611.mp3
https://www.artic.edu/assets/bebc4218-ec27-d5e5-3755-3ab79420a5d1
Sound saved to 103347.mp3
error encountered, skipping...
error encountered, skipping...
error encountered, skipping...
error encountered, skipping...
https://www.artic.edu/assets/34302929-1430-aa1c-8cca-23d94444ba2d
Sound saved to 105105.mp3
error encountered, s